# Домашнее задание 6. Автоматизация расчетов с помощью Apache AirFlow

## 1. DAG: ежедневная загрузка 4 файлов в 4 таблицы

DAG был написан и протестирован локально в docker окружении [`docker-compose.yaml`](./docker-compose.yaml). 

Файл с DAG [`process_data_daily.py`](././airflow/src/dags/process_data_daily.py).

Данные таски были реализованы в рамках `DataCollector` класса, результат выполнения:

<div style="text-align: center;">
    <img src="./assets/screenshot_data_collected.png" width="50%">
</div>

Полный DAG: 

<div style="text-align: center;">
    <img src="./assets/screenshot_full_dag.png" width="80%">
</div>

## 2. Выполнение двух запросов с сохранением в файлы

Запросы:

- Найти имена и фамилии клиентов с ТОП-3 минимальной и ТОП-3 максимальной суммой транзакций за весь период (учесть клиентов, у которых нет заказов).

- Найти ТОП-5 клиентов (по общему доходу) в каждом сегменте благосостояния (wealth_segment). Вывести имя, фамилию, сегмент и общий доход. Если в сегменте менее 5 клиентов, вывести всех.

Мы уже реализовывали в рамках дз 3. Поэтому просто возьмем их от туда и выполним в рамках тасок выполнения запросов, они реализованы с помощью `QueryRunner` класса.

- [`top_3_min_3_profit.sql`](./airflow/sql/top_3_min_3_profit.sql)
- [`top_5_wealth_segment.sql`](./airflow/sql/top_5_wealth_segment.sql)

Результат выполнения тасок сохраняется в директорию `output/` она в .gitignore.

<div style="text-align: center;">
    <img src="./assets/screenshot_output_example.png" width="50%">
</div>

Для каждой задачи формируется отдельная директория, с уникальным индентификатором выполняемого dag. В каждой директории по 2 csv файла.


## 3. Проверить, что запросы из пункта выше не вернули нулевое количество строк

Данная проверка реализована в рамках `check_queries_result` функции.

Для тестирования отправки сообщения в тестовом режиме использовался `mailhog` docker сервис. Пример успешного сообщения с нотификацией об неуспешной проверке:

<div style="text-align: center;">
    <img src="./assets/screenshot_mailhog_inbox.png" width="50%">
</div>

Скриншот логов таски с успешной попыткой посылки письма:

<div style="text-align: center;">
    <img src="./assets/screenshot_check_queries_result_log.png" width="50%">
</div>

## 4. Вывести сообщение об успешном или неуспешном выполнении DAG

Сообщение выводится в рамках `end_dag_task` таски, она помечена как `trigger_rule=TriggerRule.ALL_DONE` чтобы выполняться всегда независимо от ошибок в upstream нодах.

Если хотя бы одна таска завершилась со статусом failed, то будет залоггирован failed, иначе success:

<div style="text-align: center;">
    <img src="./assets/screenshot_end_dag_task_log.png" width="50%">
</div>

Если завершилась с failed:

<div style="text-align: center;">
    <img src="./assets/screenshot_dag_failed.png" width="45%">
    <img src="./assets/screenshot_end_dag_task_log_failed.png" width="45%">
</div>